<a href="https://colab.research.google.com/github/ofoofoo/dynamic-token-merging/blob/main/merge_sequence_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install ptflops
!pip install fvcore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=r

In [2]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
from tqdm import tqdm
from ptflops import get_model_complexity_info

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import random
import time
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
import torch
from fvcore.nn import FlopCountAnalysis

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)

ds = load_dataset("abisee/cnn_dailymail", "3.0.0")
test_dataset = ds['test']

subset_size = 100
random_indices = random.sample(range(len(test_dataset)), subset_size)
test_dataset = test_dataset.select(random_indices)
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')
total_flops = 0
total_time = 0
outputs = []
targets = []

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [17]:
print("Vocab Size", model.config.vocab_size)
print("Hidden Size", model.config.hidden_size)
print(model.lm_head)

Vocab Size 50264
Hidden Size 1024
Linear(in_features=1024, out_features=50264, bias=False)


In [67]:
# for example in tqdm(test_dataset):
for i in range(1):
    example = test_dataset[27]
    article = example['article']
    reference_summary = example['highlights']
    tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids_tensor = torch.tensor(token_ids).to(device)
    # print(token_ids_tensor.shape)

    with torch.no_grad():
        token_embeddings = model.model.shared(token_ids_tensor)
        encoder_hidden_states = token_embeddings.unsqueeze(dim=0)
        # print(token_embeddings.shape)

        encoder_outputs = model.model.encoder(
            inputs_embeds=encoder_hidden_states,
            attention_mask=None,
            output_attentions=True,
            output_hidden_states=True
        )

        '''
        num_pruning_steps = 3
        for i in range(num_pruning_steps):
            for j in range(12 // num_pruning_steps):
                result = model.model.encoder.layers[i * (12 // num_pruning_steps) + j](encoder_hidden_states,
                                                               attention_mask = None,
                                                               layer_head_mask=None,
                                                               output_attentions=True)
                encoder_hidden_states = result[0]
                attention = result[1]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, 0)
            mask = avg_attention > threshold
            encoder_hidden_states = encoder_hidden_states[mask].unsqueeze(dim=0)
        '''

        decoder_input_ids = tokenizer.encode("<s>", return_tensors="pt")
        for _ in range(30):
            decoder_outputs = model.model.decoder(
                input_ids = decoder_input_ids,
                encoder_hidden_states = encoder_hidden_states
            )
            logits = model.lm_head(decoder_outputs[0])
            # print(logits)
            predicted_token_id = logits.argmax(dim=-1)[:, -1]
            # print(predicted_token_id)
            decoder_input_ids = torch.cat([decoder_input_ids, predicted_token_id.unsqueeze(dim=0)], dim=-1)

        # print(decoder_input_ids)
        generated_summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        real_generated_tokens = model.generate(
            input_ids=token_ids_tensor.unsqueeze(dim=0),
            attention_mask=None
        )
        real_generated_summary = tokenizer.decode(real_generated_tokens[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        print(generated_summary)
        print('----------------')
        print(real_generated_summary)
        print('----------------')
        print(reference_summary)
    assert False

 of the world, the world is the world of the world. The world of the world is the world of the world. The world of the world
----------------
Ava Gardner called 34 Ennismore Gardens her 'little London retreat' and now a slice of it could be yours for an eye-watering £500,000 per year. The Hollywood siren called the building, just a short walk from Harrods in London's exclusive Knightsbridge, home until her death in 1990. Now this newly refurbished five bedroom, five bathroom apartment gives renters the chance to walk in the footsteps of the legendary screen star.
----------------
Five bed apartment in Hollywood siren's former building in Knightsbridge available to rent for £10,500 per week .
Spacious luxury flat features five bathrooms, two large reception rooms, a study and a modern kitchen .
Gardner moved to London in 1968 and made number 34 Ennismore Gardens her primary residency until her death .
She once described the home as her 'little London retreat' and spoke of her love of it

AssertionError: 

In [41]:
for example in tqdm(test_dataset):
    article = example['article']
    reference_summary = example['highlights']
    tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids_tensor = torch.tensor(token_ids).to(device)

    with torch.no_grad():
        token_embeddings = model.model.shared(token_ids_tensor)

    importance_scores = torch.norm(token_embeddings, dim=1)

    threshold = torch.quantile(importance_scores, 0.25)
    mask = importance_scores > threshold
    pruned_token_ids = token_ids_tensor[mask]

    if pruned_token_ids.numel() == 0:
        pruned_token_ids = token_ids_tensor  # Fall back to original tokens

    inputs = {
        'input_ids': pruned_token_ids.unsqueeze(0).to(device),
        'attention_mask': torch.ones_like(pruned_token_ids).unsqueeze(0).to(device)
    }
    print("attention mask shape", inputs['attention_mask'].shape)

    # flops_analysis = FlopCountAnalysis(model, inputs['input_ids'])
    # total_flops += flops_analysis.total()

    start_time = time.time()

    with torch.no_grad():
        encoder_hidden_states = token_embeddings.unsqueeze(dim=0)
        for i in range(4):
            for i in range(3):
                result = model.model.encoder.layers[i](encoder_hidden_states, attention_mask = None, layer_head_mask=None, output_attentions=True)
                encoder_hidden_states = result[0]
                attention = result[1]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, 0.2)
            mask = avg_attention > threshold
            encoder_hidden_states = encoder_hidden_states[mask].unsqueeze(dim=0)
        print(encoder_hidden_states.shape)

        decode_ids = tokenizer.encode("<s>", return_tensors="pt")
        for _ in range(10):
            print(decode_ids.shape)
            seq_length = decode_ids.shape[1]
            # causal_mask = torch.tril(torch.ones((seq_length, seq_length), device=device)).unsqueeze(0)
            # causal_mask = causal_mask.masked_fill(causal_mask == 0, float('-inf')).masked_fill(causal_mask == 1, 0.0)
            decoder_outputs = model.model.decoder(input_ids = decode_ids, encoder_hidden_states = encoder_hidden_states)
            logits = model.lm_head(decoder_outputs[0])
            next_token_id = torch.argmax(logits, dim=-1)
            print(logits, logits.shape)
            print(next_token_id)
            decode_ids = torch.cat([decode_ids, next_token_id], dim=-1)
        print(decode_ids)
        # causal_mask = torch.tril(torch.ones((seq_length, seq_length), device=device)).unsqueeze(0)
        #result = model.model.decoder(encoder_hidden_states)
        #print(result)
    assert False

    summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask']
    )

    end_time = time.time()
    print(summary_ids, summary_ids.shape)

    generated_summary = tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    print(generated_summary)

    assert False
    total_time += (end_time - start_time)
    outputs.append(generated_summary)
    targets.append(reference_summary)

average_flops = total_flops / len(test_dataset)
average_time = total_time / len(test_dataset)

rouge_results = rouge.compute(predictions=outputs, references=targets)
bleu_results = bleu.compute(predictions=outputs, references=targets)

print("Inference Results:")
print(f"Average FLOPs: {average_flops:.2e}")
print(f"Average forward pass time: {average_time:.4f} seconds")
print("ROUGE Scores:", rouge_results)
print("BLEU Score:", bleu_results)


  0%|          | 0/100 [00:00<?, ?it/s]

attention mask shape torch.Size([1, 751])
torch.Size([1, 416, 1024])
torch.Size([1, 3])
tensor([[[11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290]]]) torch.Size([1, 3, 50264])
tensor([[0, 0, 0]])
torch.Size([1, 6])
tensor([[[11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290]]]) torch.Size([1, 6, 50264])
tensor([[0, 0, 0, 0, 0, 0]])
torch.Size([1, 12])
tensor([[[11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.088

  0%|          | 0/100 [00:12<?, ?it/s]


KeyboardInterrupt: 